In [8]:
import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F


from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

##### 
# HELPER FUNCTIONS 
#####

# This function takes an image of a floor plan and a target class as input and returns a binary mask with 1's at the locations where the target class is present in the image
def isolate_class(rooms, CLASS: int):

    # Create a zero-filled numpy array with the same shape as the input image
    template = np.zeros_like(rooms)

    # Get the row and column indices of the pixels where the target class is present
    rows, cols = np.where(rooms == CLASS)

    # Set the corresponding pixels in the template to 1
    template[rows, cols] = 1
    
    # Return the binary mask
    return template

# Define a list of room classes considered as bad and good classes
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]

# This function takes an image and a list of significant nodes (i.e., classes) as input and returns the contours of the rooms, the contours of the doors, and the centroid locations of the significant nodes
def vis_nodes(img, significant_nodes):
    #signficant nodes exclude rooms we don't care about
    nodes = {}
    room_contours={}
    #door_contours={}
    for c in significant_nodes:
        nodes[c] = []
        
        # Initialize an empty list to hold the contours of the rooms of the current class
        room_contours[c] = []
        
        # Get a binary mask with 1's at the locations where the current class is present in the image
        t = isolate_class(img, c)
        
        # Find the contours of the connected components in the binary mask
        contours, _ = cv2.findContours(t.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        
        # Iterate over each contour and add it to the list of room contours and compute the centroid of the contour and add it to the list of node locations
        for s in contours:
            room_contours[c].append(s)
            nodes[c].append(np.squeeze(np.array(s), 1).mean(0))
    template = img.copy()
    
    # Return the room contours, door contours, and node locations
    return(room_contours, room_contours[12], nodes)

# This function takes the contours of the rooms and doors as input and returns the connections between the rooms as a list of pairs of room indices and as a list of pairs of room centroid locations
def get_edges(img, room_contours, door_contours):
    # Initialize empty lists to hold the room connections as indices and as centroid locations
    connections_int = []
    connections_vis = []
    
    # Iterate over each room contour to compare with other room contours
    for i, room1 in enumerate(room_contours):
        # Check that the first room contour has at least 4 points (i.e., is not a line or a point)
        if len(room1) < 4:
            return -1
        
        # Convert room contour to numpy array and create a shapely Polygon object
        room1_arr = np.array(room1).squeeze(1)
        room1_ply = Polygon(room1_arr).buffer(1)
        
        # Iterate over the remaining room contours
        for j, room2 in enumerate(room_contours[i+1:], start=i+1):
            # Check that the second room contour has at least 4 points (i.e., is not a line or a point)
            if len(room2) < 4:
                return -1
            
            # Convert room contour to numpy array and create a shapely Polygon object
            room2_arr = np.array(room2).squeeze(1)
            room2_ply = Polygon(room2_arr).buffer(1)
            
            # Check if the polygons intersect
            if room1_ply.intersects(room2_ply):
                # If the polygons intersect, add the pair of room indices to the list of connections
                connections_int.append([i, j])
                # Add the pair of room centroids to the list of connection locations
                connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
                
            else:
                # Iterate over the door contours to check if there is a door between the two rooms
                for door in door_contours:
                    # Convert door contour to numpy array and create a shapely Polygon object
                    door_arr = np.array(door).squeeze(1)
                    door_ply = Polygon(door_arr).buffer(1)
                    
                    # Check if the door intersects both rooms
                    if room1_ply.intersects(door_ply) and room2_ply.intersects(door_ply):
                        # If there is a door between the two rooms, add the pair of room indices to the list of connections
                        connections_int.append([i, j])
                        # Add the pair of room centroids to the list of connection locations
                        connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
    
    # Return the list of connections as indices and as centroid locations
    return connections_int, connections_vis


rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = '../data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

# This function takes a file path as input and returns embeddings, embeddings2, and Y.
def process_file(file_path):
    adj_list=[]
    feat_list=[]
    label_list=[]
    
    with open(file_path, 'rb') as f:
        file=pickle.load(f)
    embeddings=None
    Y=None
    
    for index, floorplan in file.items():
        if 11 not in set(floorplan.flatten()): #no undefined rooms when training
            icons = normal_set[index]['label'][1].numpy()
            rows, column = np.where(icons == 2)
            try:
                floorplan[rows, column] = 12
            except:
                continue

            # Get the contours of the rooms, doors, and nodes in the floorplan
            rooms, doors, nodes = vis_nodes(floorplan, good)
            
            rc=[]
            areas=[]
            positions=[]
            for k in rooms.keys():
                if k!=12: #not a door
                    rc+=rooms[k]
                    for count, c in enumerate(rooms[k]):
                        area=cv2.contourArea(c)
                        areas.append(area)
                        positions.append(np.array(c).squeeze(1).mean(0).tolist())
            
            try:
                idx, vis = get_edges(floorplan, rc, doors)
            except:
                get_edges(floorplan, rc, doors)==-1
                continue

            if not idx:
                continue

            nodes_lst = []
            
            for k in rooms.keys():
                if k!=12:
                    nodes_lst += ([k] * len(rooms[k]))
            nodes_lst_updated = []
            
            areas_updated=[]
            room_type_areas=[]
            for i in range(len(nodes_lst)):
                edges = set(np.array(idx).flatten())
                if i in edges:
                    nodes_lst_updated.append(nodes_lst[i])
                    areas_updated.append(areas[i])
            nodes_lst=nodes_lst_updated
            areas=areas_updated
            total_area=sum(areas)
            
            #feature #1 relative areas
            relative_areas=np.array(areas)/total_area
            
            node_attrs = {}
            for i, n in enumerate(nodes_lst):
                node_attrs[i] = room_classes[n]
            
            # Create a NetworkX graph object from the edges
            G = nx.Graph(idx)
            
            # Compute the adjacency matrix of the graph
            A = nx.adjacency_matrix(G)
            
            # calculate the degree of each node
            #feature #3 number of adjacent rooms
            degree_list = list(dict(G.degree(G.nodes())).values())
            
            
            # Convert the node classes to one-hot vectors and concatenate them with the relative areas
            X = F.one_hot(torch.tensor(nodes_lst), 11).numpy()

            print(areas)
            print(relative_areas)
            print(degree_list)
            print(nodes_lst)
            print("done with floorplan \n\n")
            break
            
            if np.isnan(areas).any() or np.isnan(relative_areas).any() or np.isnan(room_type_areas).any():
                continue
                

            X_with_areas=np.hstack(( np.array(relative_areas).reshape(-1, 1),np.array(areas).reshape(-1,1), np.reshape(degree_list, (len(degree_list), 1))))
            
            try:
                X_all=np.concatenate(([X_all, X_with_areas]))
            except:
                X_all=X_with_areas
                
            H=A@X_with_areas
            if embeddings is None:
                embeddings=H
            else:
                embeddings = np.concatenate(([embeddings , H ]), axis=0)
            
            if Y is None:
                Y=X
            else:
                Y=np.concatenate(([Y, X]), axis=0)
                
            adj_list.append(idx)
            feat_list.append(X_all)
            label_list.append(Y)

    room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]
    column_names = ['Relative Area', 'Area',  'Number of neighboring rooms']
    df = pd.DataFrame(data=embeddings, columns=column_names)
    
    return embeddings, Y, df, X_all, adj_list, feat_list, label_list #X_all is what can be used for the non-graph model.


/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:591: UserWarning: Trying to register the cmap 'rooms' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:597: UserWarning: Trying to register the cmap 'icons' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:605: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:611: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)


In [9]:
process_file("/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/dataframes/val_modified_1.pkl")


[18455.5, 41055.5, 13277.5, 7272.0]
[0.23051942 0.51280594 0.16584333 0.09083131]
[1, 2, 2, 1]
[3, 4, 5, 7]
done with floorplan 




UnboundLocalError: local variable 'X_all' referenced before assignment

In [ ]:
#MODIFIED FOR USE IN STELLARGRAPH
import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F


from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

##### 
# HELPER FUNCTIONS 
#####

# This function takes an image of a floor plan and a target class as input and returns a binary mask with 1's at the locations where the target class is present in the image
def isolate_class(rooms, CLASS: int):

    # Create a zero-filled numpy array with the same shape as the input image
    template = np.zeros_like(rooms)

    # Get the row and column indices of the pixels where the target class is present
    rows, cols = np.where(rooms == CLASS)

    # Set the corresponding pixels in the template to 1
    template[rows, cols] = 1
    
    # Return the binary mask
    return template

# Define a list of room classes considered as bad and good classes
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]

# This function takes an image and a list of significant nodes (i.e., classes) as input and returns the contours of the rooms, the contours of the doors, and the centroid locations of the significant nodes
def vis_nodes(img, significant_nodes):
    #signficant nodes exclude rooms we don't care about
    nodes = {}
    room_contours={}
    #door_contours={}
    for c in significant_nodes:
        nodes[c] = []
        
        # Initialize an empty list to hold the contours of the rooms of the current class
        room_contours[c] = []
        
        # Get a binary mask with 1's at the locations where the current class is present in the image
        t = isolate_class(img, c)
        
        # Find the contours of the connected components in the binary mask
        contours, _ = cv2.findContours(t.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        
        # Iterate over each contour and add it to the list of room contours and compute the centroid of the contour and add it to the list of node locations
        for s in contours:
            room_contours[c].append(s)
            nodes[c].append(np.squeeze(np.array(s), 1).mean(0))
    template = img.copy()
    
    # Return the room contours, door contours, and node locations
    return(room_contours, room_contours[12], nodes)

# This function takes the contours of the rooms and doors as input and returns the connections between the rooms as a list of pairs of room indices and as a list of pairs of room centroid locations
def get_edges(img, room_contours, door_contours):
    # Initialize empty lists to hold the room connections as indices and as centroid locations
    connections_int = []
    connections_vis = []
    
    # Iterate over each room contour to compare with other room contours
    for i, room1 in enumerate(room_contours):
        # Check that the first room contour has at least 4 points (i.e., is not a line or a point)
        if len(room1) < 4:
            return -1
        
        # Convert room contour to numpy array and create a shapely Polygon object
        room1_arr = np.array(room1).squeeze(1)
        room1_ply = Polygon(room1_arr).buffer(1)
        
        # Iterate over the remaining room contours
        for j, room2 in enumerate(room_contours[i+1:], start=i+1):
            # Check that the second room contour has at least 4 points (i.e., is not a line or a point)
            if len(room2) < 4:
                return -1
            
            # Convert room contour to numpy array and create a shapely Polygon object
            room2_arr = np.array(room2).squeeze(1)
            room2_ply = Polygon(room2_arr).buffer(1)
            
            # Check if the polygons intersect
            if room1_ply.intersects(room2_ply):
                # If the polygons intersect, add the pair of room indices to the list of connections
                connections_int.append([i, j])
                # Add the pair of room centroids to the list of connection locations
                connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
                
            else:
                # Iterate over the door contours to check if there is a door between the two rooms
                for door in door_contours:
                    # Convert door contour to numpy array and create a shapely Polygon object
                    door_arr = np.array(door).squeeze(1)
                    door_ply = Polygon(door_arr).buffer(1)
                    
                    # Check if the door intersects both rooms
                    if room1_ply.intersects(door_ply) and room2_ply.intersects(door_ply):
                        # If there is a door between the two rooms, add the pair of room indices to the list of connections
                        connections_int.append([i, j])
                        # Add the pair of room centroids to the list of connection locations
                        connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
    
    # Return the list of connections as indices and as centroid locations
    return connections_int, connections_vis

def create_dataframes(edges, embeddings, nodes_lst, areas, relative_areas, degree_list, room_classes_names):
    # Create the first DataFrame
    edges_df = pd.DataFrame(edges, columns=['source', 'target'])
    
    # Create the second DataFrame
    data = {
        'Area': areas,
        'Relative Area': relative_areas,
        'Number of neighboring rooms': degree_list,
        'Room Type': [room_classes_names[node] for node in nodes_lst]
    }
    attributes_df = pd.DataFrame(data)
    
    return edges_df, attributes_df


rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = '../data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

# This function takes a file path as input and returns embeddings, embeddings2, and Y.
def process_file(file_path):
    adj_list=[]
    feat_list=[]
    label_list=[]
    
    with open(file_path, 'rb') as f:
        file=pickle.load(f)
    embeddings=None
    Y=None
    
    for index, floorplan in file.items():
        if 11 not in set(floorplan.flatten()): #no undefined rooms when training
            icons = normal_set[index]['label'][1].numpy()
            rows, column = np.where(icons == 2)
            try:
                floorplan[rows, column] = 12
            except:
                continue

            # Get the contours of the rooms, doors, and nodes in the floorplan
            rooms, doors, nodes = vis_nodes(floorplan, good)
            
            rc=[]
            areas=[]
            positions=[]
            for k in rooms.keys():
                if k!=12: #not a door
                    rc+=rooms[k]
                    for count, c in enumerate(rooms[k]):
                        area=cv2.contourArea(c)
                        areas.append(area)
                        positions.append(np.array(c).squeeze(1).mean(0).tolist())
            
            try:
                idx, vis = get_edges(floorplan, rc, doors)
            except:
                get_edges(floorplan, rc, doors)==-1
                continue

            if not idx:
                continue

            nodes_lst = []
            
            for k in rooms.keys():
                if k!=12:
                    nodes_lst += ([k] * len(rooms[k]))
            nodes_lst_updated = []
            
            areas_updated=[]
            room_type_areas=[]
            for i in range(len(nodes_lst)):
                edges = set(np.array(idx).flatten())
                if i in edges:
                    nodes_lst_updated.append(nodes_lst[i])
                    areas_updated.append(areas[i])
            nodes_lst=nodes_lst_updated
            areas=areas_updated
            total_area=sum(areas)
            
            #feature #1 relative areas
            relative_areas=np.array(areas)/total_area
            
            node_attrs = {}
            for i, n in enumerate(nodes_lst):
                node_attrs[i] = room_classes[n]
            
            # Create a NetworkX graph object from the edges
            G = nx.Graph(idx)
            
            # Compute the adjacency matrix of the graph
            A = nx.adjacency_matrix(G)
            
            # calculate the degree of each node
            #feature #3 number of adjacent rooms
            degree_list = list(dict(G.degree(G.nodes())).values())
            
            # Convert the node classes to one-hot vectors and concatenate them with the relative areas
            X = F.one_hot(torch.tensor(nodes_lst), 11).numpy()

            if np.isnan(areas).any() or np.isnan(relative_areas).any() or np.isnan(room_type_areas).any():
                continue
                

            X_with_areas=np.hstack(( np.array(relative_areas).reshape(-1, 1),np.array(areas).reshape(-1,1), np.reshape(degree_list, (len(degree_list), 1))))
            
            try:
                X_all=np.concatenate(([X_all, X_with_areas]))
            except:
                X_all=X_with_areas
                
            H=A@X_with_areas
            if embeddings is None:
                embeddings=H
            else:
                embeddings = np.concatenate(([embeddings , H ]), axis=0)
            
            if Y is None:
                Y=X
            else:
                Y=np.concatenate(([Y, X]), axis=0)
                
            adj_list.append(idx)
            feat_list.append(X_all)
            label_list.append(Y)

    room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]
    column_names = ['Relative Area', 'Area',  '# of Neighboring Rooms', 'Room Type']
    df = pd.DataFrame(data=embeddings, columns=column_names)
    
    return embeddings, Y, df, X_all, adj_list, feat_list, label_list #X_all is what can be used for the non-graph model.


In [19]:
embeddings, Y, df, X_all, adj_list, feat_list, label_list=process_file("/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/dataframes/val_modified_1.pkl")
import pickle

with open('/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/ignore_outputs/data.pkl', 'wb') as f:
    pickle.dump((embeddings, Y, df, X_all, adj_list, feat_list, label_list), f)



# Stellargraph

In [13]:
#Stellargraph modification 2

import pandas as pd
import pickle
import os
import sys
import numpy as np
import cv2
import torch
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from shapely.geometry import Polygon
from skimage import transform
from torch.utils.data import DataLoader
import torch.nn.functional as F


from floortrans.models import get_model
from floortrans.loaders import (
    FloorplanSVG,
    DictToTensor,
    Compose,
    RotateNTurns
)
from floortrans.plotting import (
    segmentation_plot,
    polygons_to_image,
    draw_junction_from_dict,
    discrete_cmap
)
from floortrans.post_prosessing import (
    split_prediction,
    get_polygons,
    split_validation
)
from mpl_toolkits.axes_grid1 import AxesGrid

discrete_cmap()

os.environ['PYTHONPATH'] = '/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git:' + os.environ.get('PYTHONPATH', '')

##### 
# HELPER FUNCTIONS 
#####

# This function takes an image of a floor plan and a target class as input and returns a binary mask with 1's at the locations where the target class is present in the image
def isolate_class(rooms, CLASS: int):

    # Create a zero-filled numpy array with the same shape as the input image
    template = np.zeros_like(rooms)

    # Get the row and column indices of the pixels where the target class is present
    rows, cols = np.where(rooms == CLASS)

    # Set the corresponding pixels in the template to 1
    template[rows, cols] = 1
    
    # Return the binary mask
    return template

# Define a list of room classes considered as bad and good classes
bad=[0, 1, 2, 8, 11]
good=[3,4,5,6,7,9,10,12]

# This function takes an image and a list of significant nodes (i.e., classes) as input and returns the contours of the rooms, the contours of the doors, and the centroid locations of the significant nodes
def vis_nodes(img, significant_nodes):
    #signficant nodes exclude rooms we don't care about
    nodes = {}
    room_contours={}
    #door_contours={}
    for c in significant_nodes:
        nodes[c] = []
        
        # Initialize an empty list to hold the contours of the rooms of the current class
        room_contours[c] = []
        
        # Get a binary mask with 1's at the locations where the current class is present in the image
        t = isolate_class(img, c)
        
        # Find the contours of the connected components in the binary mask
        contours, _ = cv2.findContours(t.astype(np.uint8), mode=cv2.RETR_EXTERNAL, method=cv2.CHAIN_APPROX_NONE)
        
        # Iterate over each contour and add it to the list of room contours and compute the centroid of the contour and add it to the list of node locations
        for s in contours:
            room_contours[c].append(s)
            nodes[c].append(np.squeeze(np.array(s), 1).mean(0))
    template = img.copy()
    
    # Return the room contours, door contours, and node locations
    return(room_contours, room_contours[12], nodes)

# This function takes the contours of the rooms and doors as input and returns the connections between the rooms as a list of pairs of room indices and as a list of pairs of room centroid locations
def get_edges(img, room_contours, door_contours):
    # Initialize empty lists to hold the room connections as indices and as centroid locations
    connections_int = []
    connections_vis = []
    
    # Iterate over each room contour to compare with other room contours
    for i, room1 in enumerate(room_contours):
        # Check that the first room contour has at least 4 points (i.e., is not a line or a point)
        if len(room1) < 4:
            return -1
        
        # Convert room contour to numpy array and create a shapely Polygon object
        room1_arr = np.array(room1).squeeze(1)
        room1_ply = Polygon(room1_arr).buffer(1)
        
        # Iterate over the remaining room contours
        for j, room2 in enumerate(room_contours[i+1:], start=i+1):
            # Check that the second room contour has at least 4 points (i.e., is not a line or a point)
            if len(room2) < 4:
                return -1
            
            # Convert room contour to numpy array and create a shapely Polygon object
            room2_arr = np.array(room2).squeeze(1)
            room2_ply = Polygon(room2_arr).buffer(1)
            
            # Check if the polygons intersect
            if room1_ply.intersects(room2_ply):
                # If the polygons intersect, add the pair of room indices to the list of connections
                connections_int.append([i, j])
                # Add the pair of room centroids to the list of connection locations
                connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
                
            else:
                # Iterate over the door contours to check if there is a door between the two rooms
                for door in door_contours:
                    # Convert door contour to numpy array and create a shapely Polygon object
                    door_arr = np.array(door).squeeze(1)
                    door_ply = Polygon(door_arr).buffer(1)
                    
                    # Check if the door intersects both rooms
                    if room1_ply.intersects(door_ply) and room2_ply.intersects(door_ply):
                        # If there is a door between the two rooms, add the pair of room indices to the list of connections
                        connections_int.append([i, j])
                        # Add the pair of room centroids to the list of connection locations
                        connections_vis.append([room1_arr.mean(0), room2_arr.mean(0)])
    
    # Return the list of connections as indices and as centroid locations
    return connections_int, connections_vis


def create_dataframes(A, embeddings, nodes_lst, areas, relative_areas, degree_list, room_classes_names):
    # Create the first DataFrame
    source, target = np.where(A.todense() == 1)
    edges = np.column_stack((source, target))
    edges_df = pd.DataFrame(edges, columns=['source', 'target'])
    
    # Create the second DataFrame
    data = {
        'Area': areas,
        'Relative Area': relative_areas,
        'Number of neighboring rooms': degree_list,
        'Room Type': [room_classes_names[node] for node in nodes_lst]
    }
    attributes_df = pd.DataFrame(data)
    
    return edges_df, attributes_df


rot = RotateNTurns() #
room_classes = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage", "Undefined"]
icon_classes = ["No Icon", "Window", "Door", "Closet", "Electrical Applience" ,"Toilet", "Sink", "Sauna Bench", "Fire Place", "Bathtub", "Chimney"]
room_classes.append("Door")
data_folder = '../data/cubicasa5k/'
data_file = 'test.txt'
normal_set = FloorplanSVG(data_folder, data_file, format='txt', original_size=True)
data_loader = DataLoader(normal_set, batch_size=1, num_workers=0)
data_iter = iter(data_loader)
# Setup Model
model = get_model('hg_furukawa_original', 51)

n_classes = 44
split = [21, 12, 11]

# This function takes a file path as input and returns embeddings, embeddings2, and Y.
def process_file(file_path):
    adj_list=[]
    feat_list=[]
    label_list=[]
    
    with open(file_path, 'rb') as f:
        file=pickle.load(f)
    embeddings=None
    Y=None
    
    for index, floorplan in file.items():
        if 11 not in set(floorplan.flatten()): #no undefined rooms when training
            icons = normal_set[index]['label'][1].numpy()
            rows, column = np.where(icons == 2)
            try:
                floorplan[rows, column] = 12
            except:
                continue

            # Get the contours of the rooms, doors, and nodes in the floorplan
            rooms, doors, nodes = vis_nodes(floorplan, good)
            
            rc=[]
            areas=[]
            positions=[]
            for k in rooms.keys():
                if k!=12: #not a door
                    rc+=rooms[k]
                    for count, c in enumerate(rooms[k]):
                        area=cv2.contourArea(c)
                        areas.append(area)
                        positions.append(np.array(c).squeeze(1).mean(0).tolist())
            
            try:
                idx, vis = get_edges(floorplan, rc, doors)
            except:
                get_edges(floorplan, rc, doors)==-1
                continue

            if not idx:
                continue

            nodes_lst = []
            
            for k in rooms.keys():
                if k!=12:
                    nodes_lst += ([k] * len(rooms[k]))
            nodes_lst_updated = []
            
            areas_updated=[]
            room_type_areas=[]
            for i in range(len(nodes_lst)):
                edges = set(np.array(idx).flatten())
                if i in edges:
                    nodes_lst_updated.append(nodes_lst[i])
                    areas_updated.append(areas[i])
            nodes_lst=nodes_lst_updated
            areas=areas_updated
            total_area=sum(areas)
            
            #feature #1 relative areas
            relative_areas=np.array(areas)/total_area
            
            node_attrs = {}
            for i, n in enumerate(nodes_lst):
                node_attrs[i] = room_classes[n]
            
            # Create a NetworkX graph object from the edges
            G = nx.Graph(idx)
            
            # Compute the adjacency matrix of the graph
            A = nx.adjacency_matrix(G)
            
            # calculate the degree of each node
            #feature #3 number of adjacent rooms
            degree_list = list(dict(G.degree(G.nodes())).values())
            
            
            # Convert the node classes to one-hot vectors and concatenate them with the relative areas
            X = F.one_hot(torch.tensor(nodes_lst), 11).numpy()
            room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]

            edges_df, attributes_df = create_dataframes(A, embeddings, nodes_lst, areas, relative_areas, degree_list, room_classes_names)

            

            
            if np.isnan(areas).any() or np.isnan(relative_areas).any() or np.isnan(room_type_areas).any():
                continue
                

            X_with_areas=np.hstack(( np.array(relative_areas).reshape(-1, 1),np.array(areas).reshape(-1,1), np.reshape(degree_list, (len(degree_list), 1))))
            
            try:
                X_all=np.concatenate(([X_all, X_with_areas]))
            except:
                X_all=X_with_areas
                
            H=A@X_with_areas
            if embeddings is None:
                embeddings=H
            else:
                embeddings = np.concatenate(([embeddings , H ]), axis=0)
            
            if Y is None:
                Y=X
            else:
                Y=np.concatenate(([Y, X]), axis=0)
                
            adj_list.append(idx)
            feat_list.append(X_all)
            label_list.append(Y)
            
            
            print(f"edges_df\n{edges_df}")
            print(f"attributes_df\n{attributes_df}")
            print(f"X_all\n{X_all}")
            print(f"nodes_lst\n{nodes_lst}")
            print(f"Adj matrix\n{A}")
            print("done with floorplan \n\n")
            
            break

    room_classes_names = ["Background", "Outdoor", "Wall", "Kitchen", "Living Room" ,"Bed Room", "Bath", "Entry", "Railing", "Storage", "Garage"]
    column_names = ['Relative Area', 'Area',  'Number of neighboring rooms']
    df = pd.DataFrame(data=embeddings, columns=column_names)
    
    return embeddings, Y, df, X_all, adj_list, feat_list, label_list #X_all is what can be used for the non-graph model.


/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:591: UserWarning: Trying to register the cmap 'rooms' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:597: UserWarning: Trying to register the cmap 'icons' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:605: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)
/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/CubiCasa5k_git/submission/floortrans/plotting.py:611: UserWarning: Trying to register the cmap 'rooms_furu' which already exists.
  cm.register_cmap(cmap=cmap3)


In [14]:
process_file("/Users/alishakhan/Desktop/Career/Ascent Integrated Tech/task1/dataframes/val_modified_1.pkl")

edges_df
   source  target
0       0       1
1       1       0
2       1       2
3       2       1
4       2       3
5       3       2
attributes_df
      Area  Relative Area  Number of neighboring rooms    Room Type
0  18455.5       0.230519                            1      Kitchen
1  41055.5       0.512806                            2  Living Room
2  13277.5       0.165843                            2     Bed Room
3   7272.0       0.090831                            1        Entry
X_all
[[2.30519420e-01 1.84555000e+04 1.00000000e+00]
 [5.12805941e-01 4.10555000e+04 2.00000000e+00]
 [1.65843331e-01 1.32775000e+04 2.00000000e+00]
 [9.08313088e-02 7.27200000e+03 1.00000000e+00]]
nodes_lst
[3, 4, 5, 7]
Adj matrix
  (0, 1)	1
  (1, 0)	1
  (1, 2)	1
  (2, 1)	1
  (2, 3)	1
  (3, 2)	1
done with floorplan 




(array([[5.12805941e-01, 4.10555000e+04, 2.00000000e+00],
        [3.96362751e-01, 3.17330000e+04, 3.00000000e+00],
        [6.03637249e-01, 4.83275000e+04, 3.00000000e+00],
        [1.65843331e-01, 1.32775000e+04, 2.00000000e+00]]),
 array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0]]),
    Relative Area     Area  Number of neighboring rooms
 0       0.512806  41055.5                          2.0
 1       0.396363  31733.0                          3.0
 2       0.603637  48327.5                          3.0
 3       0.165843  13277.5                          2.0,
 array([[2.30519420e-01, 1.84555000e+04, 1.00000000e+00],
        [5.12805941e-01, 4.10555000e+04, 2.00000000e+00],
        [1.65843331e-01, 1.32775000e+04, 2.00000000e+00],
        [9.08313088e-02, 7.27200000e+03, 1.00000000e+00]]),
 [[[0, 1], [1, 9], [2, 9]]],
 [array([[2.30519420e-01, 1.84555000e+04, 1.0000

In [ ]:
# Load the edge and feature data for each floorplan and convert it to a StellarGraph object
for i, edge_data in enumerate(edge_data_list):
    feature_data=feature_data_list[i]
    G = nx.from_pandas_edgelist(edge_data)
    SG = sg.StellarGraph(G, node_features=feature_data)
    SG_list.append(SG)
    
# Define the GCN model
gcn_model = GCN(
    layer_sizes=[16, 16],
    activations=['relu', 'relu'],
    generator=None,
    dropout=0.5,
    input_shape=feature_data.shape[-1]
)

# Compile the model
gcn_model.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)

# Train the model on each floorplan graph
y_list = []
for SG in SG_list:
    label_binarizer = LabelBinarizer()
    y = label_binarizer.fit_transform(SG.node_features['room_type']) # One-hot encode the 'room_type' feature as the node labels
    y_list.append(y)
    gcn_model.generator = SG
    gcn_model.fit(
        SG.flow(
            node_ids=SG.nodes(),
            targets=y,
            batch_size=32
        ),
        epochs=50,
        verbose=1
    )

# Get the predicted node features for each floorplan graph
node_features_list = []
for SG in SG_list:
    node_features = gcn_model.predict(SG)
    node_features_list.append(node_features)

# Print the predicted node features for the first floorplan
print(node_features_list[0][:, 0]) # The first column corresponds to the 'x' feature

In [ ]:
import networkx as nx
import stellargraph as sg

# Convert the edge and feature data for floorplan1 to a StellarGraph object
G1 = nx.from_pandas_edgelist(edge_data1)
SG1 = sg.StellarGraph(G1, node_features=feature_data1)

# Convert the edge and feature data for floorplan2 to a StellarGraph object
G2 = nx.from_pandas_edgelist(edge_data2)
SG2 = sg.StellarGraph(G2, node_features=feature_data2)

# Convert the edge and feature data for floorplan3 to a StellarGraph object
G3 = nx.from_pandas_edgelist(edge_data3)
SG3 = sg.StellarGraph(G3, node_features=feature_data3)

from stellargraph.layer import GCN

# Define the GCN model
gcn_model = GCN(
    layer_sizes=[16, 16],
    activations=['relu', 'relu'],
    generator=None, # We'll set this later
    dropout=0.5,
    input_shape=feature_data1.shape[-1]
)

from tensorflow.keras import optimizers, losses, metrics

# Compile the model
gcn_model.compile(
    optimizer=optimizers.Adam(lr=0.01),
    loss=losses.categorical_crossentropy,
    metrics=[metrics.categorical_accuracy]
)

# Train the model on each floorplan graph
SG_list = [SG1, SG2, SG3] # Create a list of the StellarGraph objects for each floorplan
y_list = [get_node_labels1(), get_node_labels2(), get_node_labels3()] # Create a list of the node labels for each floorplan

for i, SG in enumerate(SG_list):
    y = y_list[i] # Get the node labels for the i-th floorplan
    gcn_model.generator = SG # Set the generator to the i-th floorplan
    gcn_model.fit(
        SG.flow(
            node_ids=SG.nodes(),
            targets=y,
            batch_size=32
        ),
        epochs=50,
        verbose=1
    )

# Get the predicted node features for each floorplan graph
node_features1 = gcn_model.predict(SG1)
node_features2 = gcn_model.predict(SG2)
node_features3 = gcn_model.predict(SG3)

# Print the predicted node features for the first floorplan
print(node_features1[:, 0]) # The first column corresponds to the 'x' feature


